In [8]:
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from barcode import Code128, Code39, EAN13
from barcode.writer import ImageWriter
import textwrap 
import re

csv = "../Order_Items_Export_-_2022-06-20.csv"
df = pd.read_csv(csv)
font = "../src/Kanit-Light.ttf"
def draw_multiple_line_text(image, text, font, text_color, text_start_height):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size
    y_text = text_start_height
    lines = textwrap.wrap(text, width=40)
    for line in lines:
        line_width, line_height = font.getsize(line)
        draw.text(((image_width - line_width) / 2, y_text), 
                  line, font=font, fill=text_color)
        y_text += line_height

def draw_multiple_line_text_barcode(image, text, font, text_color, text_start_height):
    draw = ImageDraw.Draw(image)
    image_width, image_height = image.size
    y_text = text_start_height
    lines = textwrap.wrap(text, width=40)
    for line in lines:
        line_width, line_height = font.getsize(line)
        draw.text(((image_width - line_width) / 5, y_text), 
                  line, font=font, fill=text_color)
        y_text += line_height

def createbarcode(sku):
    num_list = []
    try:
        for c in re.findall('[a-zA-Z]+',sku)[0]: num_list.append(str(ord(c)))
    except:pass
    str_to_num = "".join(num_list) + "".join(re.findall('[0-9]+',sku))
    i = 0
    while len(str_to_num) <13:
        str_to_num = f"{str_to_num}{i}"
        i+=1
    code = EAN13(str_to_num,writer=ImageWriter())
    code.save('Barcode_Copy')
    img = Image.open('Barcode_Copy.png')
    return img

In [15]:
#1 W8
#Barcode_Copy
#!!   เรียงตามตัวอักษร!!!!!
c = 0
width = 400     #4cm
height = 300    #3cm
fonts = ImageFont.truetype(font, size=20)
fonts_weight = ImageFont.truetype(font, size=15)
fonts_sku = ImageFont.truetype(font, size=20)
image_list = []
df.sort_values(by=['Product Name'],inplace=True)
for index, row in df.iterrows():
    product = row['Product Name']
    product_name = product.split('(')[0]
    product_weight = f"{re.findall('[0-9]+',product)[0]} กรัม"
    product_sku = row['Product SKU']
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)       
    text_color = (0,0,0)   #black
    draw_multiple_line_text_barcode(img, product_name, fonts, text_color, height*(5/10))
    draw_multiple_line_text_barcode(img, product_weight, fonts_weight, text_color, height*(6/10))
    draw_multiple_line_text_barcode(img, product_sku, fonts_sku, text_color, height*(1/20))
    code = createbarcode(product_sku)
    code = code.resize((int(width/2),int(height/4)))
    img.paste(code,(int(width*(1/50)),int(height*(2/10))))
    subloop = int(row['Line Item Quantity'])
    for copy in range(subloop): image_list.append(img.convert('RGB'))

image_list[0].save('Quantity_pages.pdf', save_all=True, append_images=image_list[1:])

In [3]:
#2 check
#Product_type
logo = Image.open("../src/logo-web.png")
width = 1240
height = 1754
head_font = ImageFont.truetype(font, size=160)
content_1_font = ImageFont.truetype(font, size=100)
content_2_font = ImageFont.truetype(font, size=60)
image_list = []
for index, row in df.iterrows():
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)
    text_color = (0,0,0)
    draw_multiple_line_text(img, row['Product SKU'], head_font, text_color, 400)
    draw_multiple_line_text(img, f"{re.findall('[0-9]+',row['Product Name'])[0]}  กรัม/แพ็ค", content_1_font, text_color, 700)
    draw_multiple_line_text(img, f"{row['Product Name']}     {row['Product Categories']}", content_2_font, text_color, 1000)
    draw_multiple_line_text(img, str(row['Line Item Quantity']), head_font, text_color, 1200)
    draw_multiple_line_text(img, "แพ็ค", content_1_font, text_color, 1400)
    img.paste(logo,(int(width/2.4),200))
    image_list.append(img.convert('RGB'))

image_list[0].save('Product_pages.pdf', save_all=True, append_images=image_list[1:])

In [4]:
#3
filter_values = ['ผัก','ผลไม้']
newdf = df.loc[~df['Product Categories'].isin(filter_values)]
width = 800
height = 600
fonts = ImageFont.truetype(font, size=20)
image_list = []
for index, row in newdf.iterrows():
    product = row['Product Name']
    product_name = product.split('(')[0]
    product_weight = f"{re.findall('[0-9]+',product)[0]} กรัม"
    product_sku = row['Product SKU']
    textl1 = f"{product_name}   Nutrition information"
    textl2 =f"SKU : {product_sku}   Nutrtient Composition per 100 g Edible Potion"
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)       
    text_color = (0,0,0)   #black
    draw_multiple_line_text(img, textl1, fonts, text_color, height*(1/9))
    draw_multiple_line_text(img, textl2, fonts, text_color, height*(2/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(3/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(4/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(5/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(6/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(7/9))
    draw_multiple_line_text(img, product_sku, fonts, text_color, height*(8/9))
    # code = createbarcode(product_sku)
    # code = code.resize((int(width/3),int(height/4)))
    # img.paste(code,(int(width*(1/3)),int(height*(1/2))))
    subloop = int(row['Line Item Quantity'])
    for copy in range(subloop): image_list.append(img.convert('RGB'))

image_list[0].save('FilVeg-Fruit_pages.pdf', save_all=True, append_images=image_list[1:])

In [4]:
#4 Check
#Durian_crate (5) #PDF
#sum แล้วหาร ใหญ่ 24  ,เล็ก 12 ,เดี่ยว
Durian_ID = [7576,7562,7564,8140,8216]    #กล่อง ,ลังเล็ก, ลังใหญ่, ก้านยาวกล่องเดี่ยว, ภูเขาไฟกล่องเดียว
boxshare = [24,12]          #ลังใหญ่ ,ลังเล็ก
durianDF = df.loc[df['Product ID'].isin(Durian_ID)]
sumbox = 0
width = 1240
height = 1754

fonts = ImageFont.truetype(font, size=90)
for index, row in durianDF.iterrows():
    if row['Product ID'] == Durian_ID[2]: sumbox += 24*int(row['Line Item Quantity'])
    elif row['Product ID'] == Durian_ID[1]: sumbox += 12*int(row['Line Item Quantity'])
    else: sumbox += 1*int(row['Line Item Quantity'])
box = { 'big':int(sumbox/boxshare[0]),
        'small':int((sumbox%boxshare[0])/boxshare[1]),
        'single':int((sumbox%boxshare[0])%boxshare[1])}
img = Image.new('RGB', (width, height), color='white')
imgDraw = ImageDraw.Draw(img)
text_color = (0,0,0)
text_start_height = height/5
text = [f"กล่องเดี่ยวทั้งหมด  {sumbox}  กล่อง",
        f"ลังใหญ่  {box['big']}  ลัง",
        f"ลังเล็ก  {box['small']}  ลัง",
        f"กล่องเดี่ยว  {box['single']}  ลัง"]
for t in text:
    draw_multiple_line_text(img, t, fonts, text_color, text_start_height)
    text_start_height += int(height/6)
img.save('Durian.pdf', save_all=True)
print(box)
print(durianDF)

{'big': 18, 'small': 1, 'single': 2}
     Product ID                              Product Name  Line Item Quantity  \
34         8216                 ทุเรียนภูเขาไฟ (500 กรัม)                   6   
44         7564  ทุเรียนแกะหมอนทอง (ลังใหญ่) (12000 กรัม)                   1   
49         7562   ทุเรียนแกะหมอนทอง (ลังเล็ก) (6000 กรัม)                   1   
50         8216                 ทุเรียนภูเขาไฟ (500 กรัม)                  15   
98         7562   ทุเรียนแกะหมอนทอง (ลังเล็ก) (6000 กรัม)                   1   
116        7564  ทุเรียนแกะหมอนทอง (ลังใหญ่) (12000 กรัม)                   1   
124        8216                 ทุเรียนภูเขาไฟ (500 กรัม)                   4   
125        7564  ทุเรียนแกะหมอนทอง (ลังใหญ่) (12000 กรัม)                   2   
163        7562   ทุเรียนแกะหมอนทอง (ลังเล็ก) (6000 กรัม)                   1   
178        7562   ทุเรียนแกะหมอนทอง (ลังเล็ก) (6000 กรัม)                   2   
181        7564  ทุเรียนแกะหมอนทอง (ลังใหญ่) (12000 กรัม)               

In [6]:
#5
#cover
width = 4*100
height = 4*75
image_list = []
fonts = ImageFont.truetype("../src/Kanit-Medium.ttf", size=90)
logo = Image.open("../src/LogoBW.png")
Max = int(input("Enter max number: "))

for i in range(Max):
    num = str(i+1)
    while len(num)<3:
        num = f"0{num}"
    img = Image.new('RGB', (width, height), color='white')
    imgDraw = ImageDraw.Draw(img)
    textWidth, textHeight = imgDraw.textsize(num, font=fonts)
    xText = (width - textWidth) / 2
    yText = (height - textHeight) / 2
    imgDraw.text((xText, yText), num, font=fonts, fill=(0, 0, 0))
    img.paste(logo,(int(width/2.7),50))
    image_list.append(img.convert('RGB'))
image_list[0].save('Amount_pages.pdf', save_all=True, append_images=image_list[1:])

In [21]:
def createbarcode(sku):
    num_list = []
    try:
        for c in re.findall('[a-zA-Z]+',sku)[0]: num_list.append(str(ord(c)))
    except:pass
    str_to_num = "".join(num_list) + "".join(re.findall('[0-9]+',sku))
    i = 0
    while len(str_to_num) <13:
        str_to_num = f"{str_to_num}{i}"
        i+=1
    code = EAN13(str_to_num,writer=ImageWriter())
    code.save('Barcode_Copy')
    img = Image.open('Barcode_Copy.png')
    return [img,str_to_num]

def readbarcode(num):
    if num[:-1] == "697970012345":return "EOF"
    num = num[:-1]
    while num[-1] != "0": num = num[:-1]
    num = num[:-1]      #detect trash digits
    chr_list = []
    alpha = textwrap.wrap(num[:-4],2)
    for c in alpha: chr_list.append(str(chr(int(c))))
    return str("".join(chr_list)+num[-4:])
a = "VB0411"
b = "DTF1420"
c = "IF0580"
a = "EOF"

In [22]:
print(a)
print(createbarcode(a)[1])
print(readbarcode(createbarcode(a)[1]))

EOF
6979700123456
EOF


In [10]:
x = createbarcode(c)[1]
print(x)
readbarcode(x)

7370058001234


'IF0580'

In [ ]:
createbarcode(b)[1]

In [ ]:
createbarcode(a)[0]

In [16]:
#7
newdf = pd.read_csv("../../orders-2022-06-20-00-16-07.csv")
order_id = set(newdf['No.'].tolist())
fonts = ImageFont.truetype(font, size=120)
image_list = []
text_color = (0,0,0)   #black
width = 1240  
height = 1754 
for order in order_id:
    barcode_order = createbarcode(str(order)).resize((int(width/1.5),int(height/7)))
    barcode_EOF = createbarcode("EOF").resize((int(width/3),int(height/9)))
    barcode_zero = createbarcode("0").resize((int(width/3),int(height/9)))
    customer_name = set(newdf.loc[newdf['No.']==order]['Customer Name'].tolist()).pop()
    img = Image.new('RGB', (width, height), color='white')
    ImageDraw.Draw(img)       
    draw_multiple_line_text(img, f"No.  {str(order)}", fonts, text_color, height*(4/10))
    draw_multiple_line_text(img, customer_name, fonts, text_color, height*(5.5/10))
    img.paste(barcode_order,(int(width*(1/6)),int(height*(2/10))))
    img.paste(barcode_EOF,(int(width*(1/6)),int(height*(8/10))))
    img.paste(barcode_zero,(int(width*(3/6)),int(height*(8/10))))
    image_list.append(img.convert('RGB'))
image_list[0].save('Customer_pages.pdf', save_all=True, append_images=image_list[1:])

In [ ]:
newdf.loc[newdf['No.']==8421]['Customer Name'][0]

In [18]:
ord('EOF')

TypeError: ord() expected a character, but string of length 3 found